# Assignment: Implement a Planner-Executor Workflow using CrewAI



---

### Objective:
This assignment focuses on building a **planner-executor workflow** using **CrewAI**, a framework for orchestrating roles-based autonomous AI agents. You will define distinct AI agents, assign them specialized tasks, and set up a crew to collaboratively generate content (e.g., a blog post or an article) based on a given topic. This will demonstrate how to break down complex problems into manageable sub-tasks and leverage multi-agent collaboration for more robust and structured outputs.

---

### Instructions:
1.  **LLM Access**: You'll need access to an LLM API. For this assignment, we'll primarily use **Google's Gemini Pro model** via the `google-generativeai` integration with CrewAI.
2.  **Environment Setup**: Install the necessary Python libraries: `pip install crewai crewai_tools google-generativeai pydantic`.
3.  **API Key**: Securely handle your API key. It's best practice to load it from an environment variable.
4.  **Jupyter Notebook**: All your code, outputs, observations, and analysis must be documented in this Jupyter Notebook.
5.  **Task Scenario**: You will create a workflow to **research and write a concise article (e.g., a blog post draft)** on a specific trending technology topic.
6.  **Analysis**: Critically evaluate the output from your CrewAI workflow and discuss the effectiveness of the planner-executor pattern.

---

## Part 1: Setup and LLM Configuration
Begin by installing necessary libraries and configuring your LLM for CrewAI.

### Task 1.1: Install Libraries and Configure LLM
Install `crewai` and `google-generativeai`, then set up your Google API key and initialize the LLM.

In [ ]:
# Install necessary libraries (if not already installed)
# !pip install crewai crewai_tools google-generativeai pydantic --quiet

import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool # Optional: for internet search if you have a Serper API key
from langchain_google_genai import ChatGoogleGenerativeAI

# --- YOUR API KEY HERE ---
# It's highly recommended to load your API key from an environment variable for security.
# For example:
# os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
# For this assignment, you can temporarily paste it directly, but be careful not to share your notebook with the key.
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY_HERE" # Replace with your actual Google API key

# Initialize the LLM (Gemini Pro)
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

# Optional: Initialize Serper Tool if you have an API key
# os.environ["SERPER_API_KEY"] = "YOUR_SERPER_API_KEY_HERE" # Replace with your Serper API key
# search_tool = SerperDevTool()

print("CrewAI environment setup complete!")

---

## Part 2: Define Agents (Roles)
Create the specialized AI agents that will form your crew. For a planner-executor workflow, you'll need at least two distinct agents.

### Task 2.1: The Planner Agent
Define an `Agent` whose role is to plan and break down complex tasks into actionable steps.

* **Role**: `Article Planner`
* **Goal**: To create a detailed, step-by-step outline for a comprehensive article on a given topic, including key sections, sub-topics, and main points.
* **Backstory**: An expert in content strategy and project management, known for meticulous planning and clear instructions.
* **LLM**: Use the `llm` initialized in Task 1.1.
* **Verbose**: Set to `True` for detailed logging.

In [ ]:
planner = Agent(
    role='Article Planner',
    goal='Create a detailed, step-by-step outline for a comprehensive article on a given topic, including key sections, sub-topics, and main points.',
    backstory=(
        "An expert in content strategy and project management, known for meticulous "
        "planning and clear instructions. Always ensures all necessary information "
        "is covered in a logical and structured manner."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=False # Planner should focus on planning, not delegating further
)

print("Planner Agent created!")

### Task 2.2: The Executor Agent
Define an `Agent` whose role is to execute the plan provided by the `Planner`.

* **Role**: `Article Writer`
* **Goal**: To write a high-quality, engaging article based on the provided detailed outline and key points.
* **Backstory**: A skilled wordsmith with a knack for transforming complex information into readable and compelling narratives. Focuses on clarity, creativity, and accuracy.
* **LLM**: Use the `llm` initialized in Task 1.1.
* **Verbose**: Set to `True` for detailed logging.

In [ ]:
executor = Agent(
    role='Article Writer',
    goal='To write a high-quality, engaging article based on the provided detailed outline and key points.',
    backstory=(
        "A skilled wordsmith with a knack for transforming complex information "
        "into readable and compelling narratives. Focuses on clarity, creativity, "
        "and accuracy. Can incorporate research findings if provided."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=False # Executor should focus on writing, not delegating
)

print("Executor Agent created!")

### Optional: The Researcher Agent
If you want to make your workflow more robust, you can add a `Researcher` agent with a tool to perform internet searches. This agent would be used by the Planner or Executor if they need external information.

In [ ]:
# Uncomment and complete if you have Serper API key and want to use search
# researcher = Agent(
#     role='Research Analyst',
#     goal='Conduct thorough research to gather factual information, statistics, and examples on specified topics.',
#     backstory=(
#         "A meticulous researcher with a keen eye for detail and accuracy. "
#         "Excels at finding relevant information quickly and summarizing it effectively."
#     ),
#     llm=llm,
#     tools=[search_tool], # Assign the search tool here
#     verbose=True,
#     allow_delegation=False
# )

# if 'researcher' in locals():
#     print("Researcher Agent created!")
# else:
#     print("Researcher Agent skipped (no Serper API key or uncommented).")

---

## Part 3: Define Tasks
Now, define the specific tasks that each agent will perform within the workflow. Crucially, the Executor's task will depend on the Planner's output.

### Task 3.1: Planning Task
Define the task for the `Planner` agent. Its output should be the input for the `Executor`.

* **Description**: "Create a detailed outline for a blog post/article about `{topic}`. The outline should include: a title, introduction section, 3-4 main body sections with specific sub-topics/points for each, and a conclusion section. The outline should be clear enough for an AI writer to follow easily."
* **Agent**: `planner`
* **Output File (Optional but Recommended)**: Save the plan to a markdown file (e.g., `article_plan.md`) to inspect it.

In [ ]:
article_topic = "The Impact of AI on Future Job Markets" # Define your chosen topic here

plan_task = Task(
    description=f"Create a detailed outline for a blog post/article about '{article_topic}'. "
                "The outline should include: a catchy title, an introduction section, "
                "3-4 main body sections with specific sub-topics/points for each, and a conclusion section. "
                "The outline should be clear enough for an AI writer to follow easily. "
                "Output the full outline including section headings and bullet points for sub-topics.",
    agent=planner,
    output_file='article_plan.md' # Save the generated plan to a file
)

print("Planning Task created!")

### Task 3.2: Execution Task
Define the task for the `Executor` agent. This task will take the output of the `Planning Task` as its input.

* **Description**: "Write a comprehensive and engaging article based on the following detailed outline: `{plan}`. Ensure the article is well-structured, informative, and around 800-1000 words. Maintain a professional yet accessible tone. Expand on all points in the outline."
* **Agent**: `executor`
* **Context**: Crucially, this task needs the output from `plan_task`. Use `context=[plan_task]`.
* **Output File**: Save the final article to a markdown file (e.g., `final_article.md`).

In [ ]:
write_task = Task(
    description=f"Write a comprehensive and engaging article based on the following detailed outline for '{article_topic}': '{{ {{ plan_task.output }} }}'. "
                "Ensure the article is well-structured, informative, and approximately 800-1000 words. "
                "Maintain a professional yet accessible tone. Expand on all points in the outline. "
                "Do not include any introductory or concluding remarks about the writing process, just the article itself.",
    agent=executor,
    context=[plan_task], # This links the output of plan_task as input to write_task
    output_file='final_article.md' # Save the generated article to a file
)

print("Execution Task created!")

---

## Part 4: Create and Run the Crew
Assemble your agents and tasks into a `Crew` and execute the workflow.

### Task 4.1: Form the Crew
Instantiate the `Crew` with your defined agents and tasks.

* **Agents**: List `planner` and `executor`.
* **Tasks**: List `plan_task` and `write_task` in the correct order.
* **Process**: Use `Process.sequential` to ensure tasks run one after another.
* **Verbose**: Set to `True` to see the detailed execution flow.

In [ ]:
project_crew = Crew(
    agents=[planner, executor],
    tasks=[plan_task, write_task],
    process=Process.sequential, # Tasks will run sequentially
    verbose=True
)

print("Crew formed!")

### Task 4.2: Execute the Crew
Run the crew and capture the final output.

In [ ]:
print("\n--- Kicking off the CrewAI Workflow! ---")
result = project_crew.kickoff(inputs={'topic': article_topic}) # Pass any initial inputs here

print("\n--- Workflow Finished! ---")
print("Final Result of the Crew:\n")
print(result)

print("\nAlso check 'article_plan.md' and 'final_article.md' files for full outputs.")

---

## Part 5: Analysis and Reflection
Examine the outputs generated by your CrewAI workflow and reflect on the planner-executor pattern.

### Task 5.1: Review the Outputs
Inspect the `article_plan.md` and `final_article.md` files (or the printed output) and answer the following questions.

* **Quality of Plan (`article_plan.md`)**:
    * Was the plan detailed and logical?
    * Did it contain all the sections and points you expected?
    * Was it easy for the `Executor` to follow? (Based on the final article)
* **Quality of Article (`final_article.md` / `result`)**:
    * Did the `Executor` successfully follow the `Planner`'s outline?
    * Is the article comprehensive, engaging, and well-structured?
    * Does it meet the word count and tone requirements?
    * Did you notice any repetitions, inconsistencies, or 'hallucinations'?
* **Collaboration**:
    * Describe how the `Planner` and `Executor` agents collaborated. How did the output of one agent influence the other?
    * Was the delegation of tasks clear and effective?

### Task 5.2: Reflection on Planner-Executor Workflow
Consider the benefits and limitations of this architectural pattern.

* **Benefits**: What are the key advantages of using a planner-executor workflow for complex tasks compared to a single-agent approach?
* **Limitations/Challenges**: What are the potential drawbacks or challenges you observed or can foresee with this pattern (e.g., dependency issues, communication overhead, error propagation)?
* **When to Use**: In what types of scenarios or applications would a planner-executor workflow be particularly well-suited?
* **Future Improvements**: If you were to enhance this workflow, what improvements would you implement (e.g., adding a 'Reviewer' agent, incorporating external tools like research tools, adding iteration/feedback loops between agents)?

---

### Submission:
* Ensure all code cells have been executed and their outputs are visible.
* All analysis and reflections are clearly written in markdown cells.
* **Attach** the generated `article_plan.md` and `final_article.md` files alongside your notebook if submitting to a platform that allows it, or copy their contents directly into markdown cells if not.
* Save your Jupyter Notebook as `[YourName]_CrewAI_Planner_Executor_Assignment.ipynb`.